In [1]:
using JuMP
using ECOS
using GLPK
using MAT
using Cbc
using SparseArrays
using DelimitedFiles
using LinearAlgebra
using MathOptInterface

In [2]:
# Get inputs
vars = matread("R5T2.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
K = vars["K"]

20

In [3]:
size_of_v_r = 13543
size_of_v_c = 100
size_of_s_r = 8399
epsilon = 2e-5
# calculate possible zero rows of V
possible_zero_v_rows = ones(size_of_v_r)
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if U[i, j] < -epsilon || L[i, j] > epsilon
            possible_zero_v_rows[i] = 0
            break
        end
    end
end
num_impossible_zero_v_rows = Int(sum(1-possible_zero_v_rows[i] for i in 1:size_of_v_r))
print(num_impossible_zero_v_rows)
println(" number of rows of V cannot be 0")

232 number of rows of V cannot be 0


In [9]:
# First, use round 2 problem to find V (first solving l1 and then iterating)
V = zeros(size_of_v_r, size_of_v_c)
for col in 1:size_of_v_c
    println(string("solving for column ########", string(col), "#####"))
    
    model = JuMP.Model(with_optimizer(ECOS.Optimizer, verbose=false)) 
    
    # define the problem
    @variable(model, v[1:size_of_v_r])
    @variable(model, v_abs[1:size_of_v_r])
    @constraint(model, v_abs .>= v)
    @constraint(model, v_abs .>= -v)
    @constraint(model, Array(L[:, col]) .<= v .<= Array(U[:, col]))  # l1 <= v <= u1
    
    for i in 1:size_of_s_r
        xs, vals = SparseArrays.findnz(S[i, :])
        @constraint(model, 0 == sum(vals[k] * v[xs[k]] for k in 1:length(vals)))
    end
    @objective(model, Min, sum(possible_zero_v_rows[i] * v_abs[i] for i in 1:size_of_v_r))  # min ||v||_1
    optimize!(model)
    println(termination_status(model))
    println(primal_status(model))
    starting_v = JuMP.value.(v)
    println("l1 problem solved. going to iterate.")
    delta = 0.002
    n = 2
    num_iteration = 5
    for iteration in 1:num_iteration
        print("doing iteration ")
        println(iteration)
        
        w = zeros(size_of_v_r)
        for i in 1:size_of_v_r
            w[i] = delta ^ n / (delta ^ n + abs(starting_v[i])^n)
        # w_i = sgn(v_i) * delta^n / (delta^n + v_i^n)
        end
        for i in 1:size_of_v_r
            if abs(starting_v[i]) < epsilon && possible_zero_v_rows[i] == 1
                @constraint(model, v[i] <= starting_v[i])
            end
        end
        @objective(model, Min, sum(possible_zero_v_rows[i] * v[i] * w[i] for i in 1:size_of_v_r))  # min ||v||_1
        optimize!(model)
        println(termination_status(model))
        println(primal_status(model))
        starting_v = JuMP.value.(v)
        
        # decrement n
        n -= 1 / num_iteration
        delta += 0.3
    end
    
    for i in 1:size_of_v_r
        V[i, col] = starting_v[i]
    end
    
end

solving for column ########1#####
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for column ########2#####
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for column ########3#####
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for column ########4#####
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing ite

In [7]:
# This is the first parts result. Lets save that!
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if abs(V[i, j]) < epsilon
            V[i, j] = 0
        end
        V[i, j] = round(V[i, j], digits=4)
    end
end
DelimitedFiles.writedlm(string("output.txt"), V, ',')

In [5]:
# Lets first find the rows of V that are already fully zero. We wont change them any further.
zero_rows_of_v = ones(size_of_v_r)
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if abs(V[i, j]) > epsilon
            zero_rows_of_v[i] = 0
        else
            V[i, j] = 0
        end
    end
end
num_zero_v_rows = Int(sum(zero_rows_of_v[i] for i in 1:size_of_v_r))

12491

In [6]:
# Now, we want to select at most k columns to replace with zeros that maximize the number of zero rows
# To solve this, we make a MILP problem
model = JuMP.Model(Cbc.Optimizer)

@variable(model, selected_cols[1:size_of_v_c])  # selected_cols[i]=1 if the ith column is going to get zero and 0 otherwise

remaining_v_rows = Int(size_of_v_r - num_impossible_zero_v_rows - num_zero_v_rows)
@variable(model, v_norm[1:remaining_v_rows]) # a binary variable indicating zero norm of rows in V 

println("start for")
v_norm_id = 1
interesting_v_rows_ids = zeros(Int, Int(size_of_v_r - num_zero_v_rows), )
for i in 1:size_of_v_r
    if zero_rows_of_v[i] == 0 && possible_zero_v_rows[i] == 1
        for j in 1:size_of_v_c
            mx_val = max(abs(L[i, j]), abs(U[i, j]))
            @constraint(model, mx_val * v_norm[v_norm_id] >= (1 - selected_cols[j]) * V[i,j])
            @constraint(model, mx_val * v_norm[v_norm_id] >= -(1 - selected_cols[j]) * V[i,j])
        end
        @constraint(model, v_norm[v_norm_id] in MathOptInterface.ZeroOne())
        v_norm_id = Int(v_norm_id+1)
    end
end
println("end for")
@constraint(model, sum(selected_cols[i] for i in 1:size_of_v_c) <= K)
for i in 1:size_of_v_c
    @constraint(model, selected_cols[i] in MathOptInterface.ZeroOne())
end
@objective(model, Min, sum(v_norm[i] for i in 1:remaining_v_rows));

start for
end for


In [7]:
# Lets see the model size
model

A JuMP Model
Minimization problem with:
Variables: 920
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 164000 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 920 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: selected_cols, v_norm

In [ ]:
# Now we solve the problem to find good columns to make zero
optimize!(model)
println(termination_status(model))
print(primal_status(model))
v_norm_ans = JuMP.value.(v_norm)
selected_cols_ans = JuMP.value.(selected_cols)

In [14]:
# Now lets make the selected columns zero in V
num_zeroed_cols = 0
for j in 1:size_of_v_c
    if abs(selected_cols_ans[j] - 1) < epsilon
        for i in 1:size_of_v_r
            if zero_rows_of_v[i] == 0 && possible_zero_v_rows[i] == 1 
                V[i, j] = 0
            end
        end
        num_zeroed_cols += 1
    end
end

In [15]:
# This is kind of a basic (but probably strong) result. Lets save that!
DelimitedFiles.writedlm(string("output.txt"), V, ',')

In [16]:
# Lets recalculate number of zero rows of the new V after making some columns zero
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if abs(V[i, j]) > epsilon
            zero_rows_of_v[i] = 0
            break
        end
    end
end
num_zero_v_rows = Int(sum(zero_rows_of_v[i] for i in 1:size_of_v_r))

12182

In [17]:
# Now we use MILP to zero out as much rows as possible
model = JuMP.Model(Cbc.Optimizer)
remaining_v_rows = Int(size_of_v_r - num_impossible_zero_v_rows - num_zero_v_rows)
@variable(model, v[1:Int(size_of_v_r - num_zero_v_rows), 1:Int(size_of_v_c-num_zeroed_cols)])  # define variable v
@variable(model, v_norm[1:remaining_v_rows])

println("start for")

v_norm_id = 1
v_i_id = 1
interesting_v_rows_ids = zeros(Int, Int(size_of_v_r - num_zero_v_rows))
for i in 1:size_of_v_r
    if zero_rows_of_v[i] == 0 && possible_zero_v_rows[i] == 1
        v_j_id = 1
        for j in 1:size_of_v_c
            if selected_cols_ans[j] == 0
                mx_val = max(abs(L[i, j]), abs(U[i, j]))
                @constraint(model, mx_val * v_norm[v_norm_id] >= v[v_i_id,v_j_id])
                @constraint(model, mx_val * v_norm[v_norm_id] >= -v[v_i_id,v_j_id])
                v_j_id = Int(v_j_id + 1)
            end
        end
        @constraint(model, v_norm[v_norm_id] in MathOptInterface.ZeroOne())
        v_norm_id = Int(v_norm_id+1)
    end
    if zero_rows_of_v[i] == 0
        interesting_v_rows_ids[Int(v_i_id)] = Int(i)
        v_i_id = Int(v_i_id + 1)
    end
end

interesting_v_cols_ids = zeros(Int, Int(size_of_v_c - num_zeroed_cols))
v_j_id = 1
for j in 1:size_of_v_c
    if selected_cols_ans[j] == 0
        interesting_v_cols_ids[Int(v_j_id)] = Int(j)
        v_j_id = Int(v_j_id + 1)
    end
end
println("endfor")

@constraint(model, Matrix(L[interesting_v_rows_ids, interesting_v_cols_ids]) .<= v .<= Matrix(U[interesting_v_rows_ids, interesting_v_cols_ids]))  # l1 <= v <= u1
modified_S = S[:, interesting_v_rows_ids]
v_j_id = 1
for j in 1:size_of_v_c
    if selected_cols_ans[j] == 0
        for i in 1:size_of_s_r
            xs, vals = SparseArrays.findnz(modified_S[i, :])
            if length(vals) > 0
                @constraint(model, 0 == sum(-vals[k] * v[xs[k], v_j_id] for k in 1:length(vals)))
            end
        end
        v_j_id = Int(v_j_id + 1)
    end
    println(j)
end

@objective(model, Min, sum(v_norm[i] for i in 1:remaining_v_rows));

start for
endfor
haha
huhu
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [18]:
model

A JuMP Model
Minimization problem with:
Variables: 37197
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 15640 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 57360 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 36480 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 717 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: v, v_norm

In [19]:
# Lets solve the above MILP problem to find locally best solution
optimize!(model)
println(termination_status(model))
print(primal_status(model))
starting_v_norm = JuMP.value.(v_norm)
starting_v = JuMP.value.(v)
sparse(starting_v)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 112.907 - 0.64 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 195 strengthened rows, 0 substitutions
Cgl0004I processed model has 3526 rows, 4617 columns (476 integer (476 of which binary)) and 22212 elements
Cbc0038I Initial state - 40 integers unsatisfied sum - 8.0911
Cbc0038I Pass   1: (1.41 seconds) suminf.    5.74311 (34) obj. 169.909 iterations 59
Cbc0038I Pass   2: (1.43 seconds) suminf.    5.30124 (31) obj. 170.301 iterations 24
Cbc0038I Pass   3: (1.47 seconds) suminf.    0.79629 (10) obj. 181.796 iterations 298
Cbc0038I Solution found of 191
Cbc0038I Relaxing continuous gives 191
Cbc0038I Before mini branch and bound, 433 integers at bound fixed and 2774 continuous
Cbc0038I Full problem 3526 rows 4617 columns, reduced to 604 rows 555 columns
Cbc0038I Mini branch and bound did not improve solution (1.65 seconds

FEASIBLE_POINTirCuts) - 154 row cuts average 22.8 elements, 0 column cuts (0 active)  in 0.054 seconds - new frequency is -100
Cbc0001I Search completed - best objective 186, took 944 iterations and 0 nodes (6.09 seconds)
Cbc0035I Maximum depth 0, 322 variables fixed on reduced cost
Cuts at root node changed objective from 168.558 to 185.674
Probing was tried 5 times and created 84 cuts of which 0 were active after adding rounds of cuts (0.026 seconds)
Gomory was tried 5 times and created 94 cuts of which 0 were active after adding rounds of cuts (0.024 seconds)
Knapsack was tried 5 times and created 23 cuts of which 0 were active after adding rounds of cuts (0.127 seconds)
Clique was tried 5 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 5 times and created 150 cuts of which 0 were active after adding rounds of cuts (0.026 seconds)
FlowCover was tried 5 times and created 0 cuts of which 0 were active after ad

912×40 SparseMatrixCSC{Float64, Int64} with 4084 stored entries:
⣢⣾
⣽⣿
⣽⢭
⢭⠽
⢿⡿
⣶⣶
⣿⣿
⣩⣋
⠿⠿
⣶⣷
⠮⠦
⣶⣶
⣿⣿
⢬⢧
⣶⣤
⠻⠛
⣿⣿
⣿⣾
⣿⣿
⣿⣿
⣿⡿
⣦⣿
⣯⣿
⣿⣯
⣾⣿
⡿⡿

In [22]:
# Now we calculate the final V and save it
for i in 1:Int(size_of_v_r - num_zero_v_rows)
    for j in 1:Int(size_of_v_c - num_zeroed_cols)
        if abs(starting_v[i, j]) < epsilon
            starting_v[i, j] = 0
        end
        V[interesting_v_rows_ids[i], interesting_v_cols_ids[j]] = starting_v[i, j]
    end
end
DelimitedFiles.writedlm(string("output.txt"), V, ',')